# node structure 

Ideas: 
1-use the category of the meal as key of the dictionary of the dataset
2- this will be used in coordination with state.meal
3- the dataset will have the following form 
    {
        category : {{object},{object2},{object3}}
    }

# Problems: 
1- we have to check for daily calories so that they're satisfied which translates to checking the calories every three levels which represents a day 

In [ ]:
import queue
from copy import deepcopy
import math
from math import sin, cos, radians, atan2, sqrt

# here for the convenience: we chose to add a meal and a day data member.
# using this we can make the check for the daily constraints easier
# for instance for calories we'll just check if the meal is a dinner so that we know that we are at the end of the day and o check if the calories are ok for the day. which we will call a valid state: a valid state is a state that will be considered a worthy state to be expanded. if a state is not valid; we simply won't expand it. ( pruning)




# ------------------------------
# 1. Node Class Definition
# ------------------------------
class Node:
    def __init__(self, state, parent=None, meal=None,day = None,Valid = [], g=0, f=0):
        self.state = state        # Current meal plan state (7x3 matrix)
        self.parent = parent      # Parent node
        self.day = day
        self.meal =meal        # Meal object (if applicable)this will signify what the current category of the state is, will be passed from parent to childre: parent: breakfast -> child: lunch 
        self.depth = 0 if parent is None else parent.depth + 1
        self.g = g                # Cumulative cost from start to this node
        self.f = f                # Evaluation cost (g + heuristic if applicable)
        self.ValidArray = Valid

    def __hash__(self):
        if isinstance(self.state, list):
            state_tuple = tuple([tuple(row) for row in self.state])
            return hash(state_tuple)
        return hash(self.state)

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __gt__(self, other):
        return isinstance(other, Node) and self.f > other.f


# class definition

In [ ]:

# ------------------------------
# 2. mealPlanning Class for Food Recommendation System
# ------------------------------

class mealPlanning:
    def __init__(self, initial_state, goal_state):
        self.initial_state = initial_state  # Empty meal plan (7x3 matrix)
        self.goal_state = goal_state        # Target meal plan with calorie and cost goals
        
        # Constraints
        self.max_repetitions = 1  # Max times a dish can be repeated in a week
        self.meals_per_day = 3    # Breakfast, Lunch, Dinner
        self.days_per_week = 7    # Monday through Sunday

    def is_goal(self, current_state):
        # cost is here and other global constraints and also the 7 validity checks of nutrients
        pass

    def _is_complete(self, state):
        # Check if all meal slots are filled
        for day in range(self.days_per_week):
            for meal in range(self.meals_per_day):
                if state[day][meal] is None:
                    return False
        return True

    def _calculate_plan_stats(self, state):
        # Calculate total calories and cost for the meal plan
        total_calories = 0
        total_cost = 0
        
        for day in range(self.days_per_week):
            for meal in range(self.meals_per_day):
                dish_id = state[day][meal]
                if dish_id is not None:
                    dish_info = self.state_transition_model[dish_id]
                    total_calories += dish_info['calories']
                    total_cost += dish_info['cost']
                    
        return total_calories, total_cost

    def get_valid_actions(self, current_node):
        #find the category of the meal (i.e breakfast ..Etc)
        meal_idx = current_node.meal
        valid_meals = []
        if meal_idx != "dinner":
            for meal in Dataset[meal]:
                if meal in current_node.state
                
        else:
        
        # For each dish in our database
        for dish_id, dish_info in self.state_transition_model.items():
            # Check if this dish would violate the repetition constraint
            if self._count_dish_occurrences(current_state, dish_id) < self.max_repetitions:
                # The action is to place this dish at the current empty slot
                valid_actions[dish_id] = dish_info['cost']
                
        return valid_actions

    def expand_node(self, node, use_cost=True, use_heuristic=False):
        state = node.state
        
        # Find valid actions for this state
        valid_actions = self.get_valid_actions(state)
        
        # Find the next empty slot
        day_idx, meal_idx = self._find_next_empty_slot(state)
        
        child_nodes = []
        for dish_id, cost in valid_actions.items():
            # Create a new state by placing the dish in the empty slot
            child_state = deepcopy(state)
            child_state[day_idx][meal_idx] = dish_id
            
            # Calculate the new g value (actual cost)
            child_g = node.g + cost
            
            # Calculate the coordinates for heuristic (for meal planning this could be nutritional balance)
            dish_info = self.state_transition_model[dish_id]
            coordinates = [dish_info['calories'], dish_info['nutritional_value']]
            
            # Calculate the total evaluation cost
            total_cost = self.get_total_cost(child_g, coordinates, use_cost, use_heuristic)
            
            # Create the child node
            child_node = Node(
                state=child_state,
                parent=node,
                action=dish_id,
                g=child_g,
                f=total_cost
            )
            
            child_nodes.append(child_node)
            
        return child_nodes

    def get_total_cost(self, g, coordinates, use_cost, use_heuristic):
        # Calculate the total cost based on actual cost and heuristic
        total = 0
        
        if use_cost:
            total += g  # Add the actual cost
            
        if use_heuristic:
            # Calculate the heuristic (distance to goal)
            calories_remaining = max(0, self.goal_state['calories'] - coordinates[0])
            nutritional_balance = self.goal_state['nutritional_value'] - coordinates[1]
            
            # Weighted combination of calories and nutritional balance
            heuristic = (calories_remaining * self.goal_state['calories_weight'] + 
                         abs(nutritional_balance) * self.goal_state['nutritional_weight'])
            
            total += heuristic
            
        return total

    def get_crow_flies_distance(self, coordinates, goal_coordinates):
        # For food recommendation, this could be nutritional distance
        # More sophisticated implementations might use a multi-dimensional distance metric
        
        # Using Euclidean distance for simplicity
        squared_diff_sum = 0
        for i in range(len(coordinates)):
            squared_diff_sum += (coordinates[i] - goal_coordinates[i]) ** 2
            
        return math.sqrt(squared_diff_sum)

    def print_node(self, node):
        print("Action:", node.action, "| Depth:", node.depth)
        if node.action is not None:
            dish_info = self.state_transition_model[node.action]
            print(f"Added dish: {dish_info['name']}, Calories: {dish_info['calories']}, Cost: {dish_info['cost']}")
        
        # Print the meal plan
        if node.state is not None:
            print("\nCurrent Meal Plan:")
            days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
            meals = ["Breakfast", "Lunch", "Dinner"]
            
            for day_idx, day in enumerate(days):
                print(f"\n{day}:")
                for meal_idx, meal in enumerate(meals):
                    dish_id = node.state[day_idx][meal_idx]
                    if dish_id is not None:
                        dish_info = self.state_transition_model[dish_id]
                        print(f"  {meal}: {dish_info['name']} ({dish_info['calories']} cal, ${dish_info['cost']})")
                    else:
                        print(f"  {meal}: ---")

# Example usage with a toy meal planning problem
def toy_test_meal_planning_problem():
    # Define a database of dishes with their properties
    dishes_database = {
        'D1': {
            'name': 'Oatmeal with Berries',
            'calories': 300,
            'cost': 3.50,
            'nutritional_value': 8,
            'category': 'breakfast'
        },
        'D2': {
            'name': 'Scrambled Eggs & Toast',
            'calories': 400,
            'cost': 4.00,
            'nutritional_value': 7,
            'category': 'breakfast'
        },
        'D3': {
            'name': 'Greek Salad',
            'calories': 350,
            'cost': 6.00,
            'nutritional_value': 9,
            'category': 'lunch'
        },
        'D4': {
            'name': 'Chicken Sandwich',
            'calories': 550,
            'cost': 7.50,
            'nutritional_value': 6,
            'category': 'lunch'
        },
        'D5': {
            'name': 'Grilled Salmon',
            'calories': 450,
            'cost': 9.00,
            'nutritional_value': 10,
            'category': 'dinner'
        },
        'D6': {
            'name': 'Vegetable Stir-Fry',
            'calories': 380,
            'cost': 5.50,
            'nutritional_value': 8,
            'category': 'dinner'
        }
    }
    
    # Define the initial state: empty 7x3 matrix (7 days, 3 meals per day)
    initial_state = [[None for _ in range(3)] for _ in range(7)]
    
    # Define the goal state: target calories and cost for the week
    goal_state = {
        'calories': 8400,  # Average 1200 calories per day
        'calories_margin': 500,  # Acceptable margin of error
        'cost': 140.00,  # Weekly food budget
        'cost_margin': 15.00,  # Acceptable margin of error
        'nutritional_value': 7.5,  # Target average nutritional value
        'calories_weight': 0.6,  # Weight for calories in the cost function
        'nutritional_weight': 0.4  # Weight for nutritional balance in the cost function
    }
    
    # Create the problem instance
    meal_plan_problem = mealPlanning(
        initial_state=initial_state,
        goal_state=goal_state,
        state_transition_model=dishes_database
    )
    
    # Test the problem methods
    print("Testing meal planning problem:")
    
    # Test is_goal method
    print("\nTesting is_goal:")
    print("Is empty plan a goal?", meal_plan_problem.is_goal(initial_state))  # Should be False
    
    # Test get_valid_actions
    print("\nTesting get_valid_actions:")
    valid_actions = meal_plan_problem.get_valid_actions(initial_state)
    print("Valid actions for initial state:", valid_actions)
    
    # Test expand_node
    print("\nTesting expand_node:")
    start_node = Node(state=initial_state, g=0)
    children = meal_plan_problem.expand_node(start_node)
    print(f"Number of child nodes: {len(children)}")
    
    # Test print_node
    print("\nTesting print_node:")
    meal_plan_problem.print_node(start_node)
    
    if children:
        print("\nSample child node:")
        meal_plan_problem.print_node(children[0])
    
    # Test partial meal plan
    print("\nTesting with partial meal plan:")
    # Create a partial meal plan (first day's meals are filled)
    partial_state = deepcopy(initial_state)
    partial_state[0][0] = 'D1'  # Monday breakfast
    partial_state[0][1] = 'D3'  # Monday lunch
    partial_state[0][2] = 'D5'  # Monday dinner
    
    partial_node = Node(state=partial_state, g=dishes_database['D1']['cost'] + dishes_database['D3']['cost'] + dishes_database['D5']['cost'])
    print("Partial meal plan:")
    meal_plan_problem.print_node(partial_node)
    
    # Test expanding the partial node
    print("\nExpanding partial node:")
    partial_children = meal_plan_problem.expand_node(partial_node)
    print(f"Number of valid actions: {len(partial_children)}")
    
    # Advanced test: Check that repetition constraints are enforced
    print("\nTesting repetition constraints:")
    repetition_test_state = deepcopy(initial_state)
    # Fill with D1 twice (should still allow D1 as a valid action)
    repetition_test_state[0][0] = 'D1'
    repetition_test_state[1][0] = 'D1'
    
    repetition_node = Node(state=repetition_test_state, g=dishes_database['D1']['cost'] * 2)
    repetition_actions = meal_plan_problem.get_valid_actions(repetition_test_state)
    
    print("State with D1 used twice:")
    meal_plan_problem.print_node(repetition_node)
    print("Valid actions include D1?", 'D1' in repetition_actions)
    
    # Now add D1 again (third time) - should not allow D1 as a valid action
    repetition_test_state[2][0] = 'D1'
    repetition_node = Node(state=repetition_test_state, g=dishes_database['D1']['cost'] * 3)
    repetition_actions = meal_plan_problem.get_valid_actions(repetition_test_state)
    
    print("\nState with D1 used three times:")
    meal_plan_problem.print_node(repetition_node)
    print("Valid actions include D1?", 'D1' in repetition_actions)  # Should be False

if __name__ == "__main__":
    # Run the toy test for meal planning
    toy_test_meal_planning_problem()